In [ ]:
# https://www.psycopg.org/docs/usage.html 
# https://www.psycopg.org/docs/module.html#psycopg2.connect

In [ ]:
import psycopg2
# # NEVER PUT a password in the code:/connection-string  used below: just for simple test/learning
conn = psycopg2.connect(dbname="dvdrental", user="testuser", password="testuser123")

print(conn)
conn.close()

In [ ]:
import psycopg2
connection_string = "dbname=dvdrental user=testuser password=testuser123"
conn = psycopg2.connect(connection_string)

print(conn)
conn.close()

In [ ]:
## ATTEMPT CONNECT TO dbname="postgres"
## SEE fails IF USER doesn't have priviliges
import psycopg2
conn = psycopg2.connect(dbname="postgres", user="testuser", password="testuser123")

try:
    curr = conn.cursor()
    curr.execute("SELECT * FROM booking;")
finally:
    conn.close() # don't forget to close

In [ ]:
## A SELECT *
import psycopg2
conn = psycopg2.connect(dbname="dvdrental", user="testuser", password="testuser123")

curr = conn.cursor()
curr.execute("SELECT * FROM staff;")
result = curr.fetchall()
for record in result:
    print(record)

curr.close()
conn.close() # don't forget to close

In [ ]:
## Putting the SQL into a String
import psycopg2
conn = psycopg2.connect(dbname="dvdrental", user="testuser", password="testuser123")

SQL = "SELECT * FROM staff;"
print(SQL)  # print SQL string

curr = conn.cursor()
curr.execute(SQL)
result = curr.fetchall()  #experiment: fetchone()
for record in result:
    print(record)
    
curr.close()
conn.close() # don't forget to close

In [ ]:
## Building an SQL String with user input
## using (e.g.) -> input() 
import psycopg2
conn = psycopg2.connect(dbname="dvdrental", user="testuser", password="testuser123")

SQL = f"SELECT * FROM {input('Enter table name: ')};"
print(SQL)  # print SQL string

curr = conn.cursor()
curr.execute(SQL)
result = curr.fetchall()
for record in result:
    print(record)
    
    
curr.close()
conn.close() # don't forget to close

In [ ]:
## DANGER!! SQL INJECTION... CARE NEEDED
## Typically NEVER build an SQL String like this...
import psycopg2
conn = psycopg2.connect(dbname="dvdrental", user="testuser", password="testuser123")

SQL = f"SELECT * FROM Staff WHERE first_name = '{input('Enter first_name: ')}';"

# DANGER ABOVE: SQL INJECTION: consider...
# SQL = f"SELECT * FROM Staff WHERE first_name = 'Mike' OR TRUE;"

print(SQL)  # print SQL string
curr = conn.cursor()
curr.execute(SQL)
result = curr.fetchall()
print(result)
    
curr.close()
conn.close() # don't forget to close

In [ ]:
## USE SQL parameters to prevent SQL INJECTION
## To prevent SQL injection attacks: parameterized queries
## (prepared statements)
import psycopg2
conn = psycopg2.connect(dbname="dvdrental", user="testuser", password="testuser123")

# Get user input
first_name = input("Enter first_name: ")

# Build the SQL query string with a placeholder for the parameter
SQL = "SELECT * FROM Staff WHERE first_name = %s"
data = (first_name, ) # pass parameter as a py single-element tuple

curr = conn.cursor()
curr.execute(SQL, data ) 

result = curr.fetchall()  #experiment: fetchone()
for record in result:
    print(record)
    
curr.close()
conn.close() # don't forget to close

## ABOVE
## curr.execute( SQL, data )
## data needs to be a Python sequence
## A Tuple is a sequence but to make a single-element tuple
## need: ( val, ) ...i.e. trailing comma

In [ ]:
## At this point: revert to psql (for example)
## and Grant update priviliges on the Staff table for the testuser
## THEN follow below...

In [ ]:
## The COMMIT concept
## Attempting an UPDATE after 
## GRANTing UPDATE priviliges...

import psycopg2
conn = psycopg2.connect(dbname="dvdrental", user="testuser", password="testuser123")

SQL_UPDATE = "UPDATE staff SET first_name = 'Michael' where first_name = 'Mike';"
print(SQL_UPDATE)  # print SQL string
curr = conn.cursor()
curr.execute(SQL_UPDATE)

#THEN
SQL_SELECT = "SELECT * FROM staff;"
print(SQL_SELECT)  # print SQL string

curr.execute(SQL_SELECT)
result = curr.fetchall()  #experiment: fetchone()
for record in result:
    print(record)
    
curr.close()
conn.close() # don't forget to close

In [ ]:
## but now try 
## psql:
## SELECT * FROM staff;

In [ ]:
## See a new connection 
## through python
## No change to the DB
import psycopg2
conn = psycopg2.connect(dbname="dvdrental", user="testuser", password="testuser123")

SQL_SELECT = "SELECT * FROM staff;"
print(SQL_SELECT)  # print SQL string

curr = conn.cursor()
curr.execute(SQL_SELECT)
result = curr.fetchall()  #experiment: fetchone()
for record in result:
    print(record)
    
curr.close()
conn.close() # don't forget to close

In [ ]:
## REPEATING ABOVE NOW
## ONLY with conn.commit();

In [ ]:
import psycopg2
conn = psycopg2.connect(dbname="dvdrental", user="testuser", password="testuser123")

SQL_UPDATE = "UPDATE staff SET first_name = 'Michael' where first_name = 'Mike';"
print(SQL_UPDATE)  # print SQL string
curr = conn.cursor()
curr.execute(SQL_UPDATE)
conn.commit()               ##ADDED conn.commit()


#THEN
SQL_SELECT = "SELECT * FROM staff;"
print(SQL_SELECT)  # print SQL string

curr.execute(SQL_SELECT)
result = curr.fetchall()  #experiment: fetchone()
for record in result:
    print(record)
    
curr.close()
conn.close() # don't forget to close

In [ ]:
## SEE
## psql:
## SELECT * FROM staff;
## DB CHANGED after 'conn.commit()'

In [ ]:
## Auto-COMMIT
## using the python 'with' syntax
## psycopg2: Connection string
## and 
## WITH SYNTAX TO AUTO COMMIT?
## https://www.psycopg.org/docs/usage.html#with-statement

## RUN a SELECT * first to see first_name currently 'Micael'
import psycopg2
connection_string = "dbname=dvdrental user=testuser password=testuser123"

SQL_UPDATE = "UPDATE staff SET first_name = 'Mike' where first_name = 'Michael';" 
print(SQL_UPDATE)  # print SQL string

with psycopg2.connect(connection_string) as conn:
    curr = conn.cursor()
    curr.execute(SQL_UPDATE)  #auto-commit()

curr.close()
conn.close()

In [ ]:
# https://www.psycopg.org/docs/usage.html#with-statement
import psycopg2
connection_string = "dbname=dvdrental user=testuser password=testuser123"
conn = psycopg2.connect(connection_string)

with conn:  #will auto-commit the conn
    with conn.cursor() as curr: #will auto-close the cursor
        curr.execute("SELECT * FROM staff")

        for record in curr.fetchall():
            print(record)

conn.close()